In [ ]:
from tqdm.notebook import tqdm
import pandas as pd
from ast import literal_eval
import numpy as np
import math

In [ ]:
df = pd.read_csv("data/movies_metadata.csv")

In [ ]:
df = df[['id','genres']]
df = df.drop([19730, 29503, 35587])
df['id'] = df['id'].astype('int')

df['genres'] = df['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
profile = []
for i in df.index:
    for g in df['genres'][i]:
        if g not in profile:
            profile.append(g)
credits = pd.read_csv("data/credits.csv")
credits['id'] = credits['id'].astype('int')
df = df.merge(credits, on='id')

def find_director(crew):
    for i in crew:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

#convert cast column into list type
df['cast'] = df['cast'].apply(literal_eval)
#extract only the name of the cast
df['cast'] = df['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
#select the first three actors
df['cast'] = df['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
#convert crew column into list
df['crew'] = df['crew'].apply(literal_eval)
#extract the name of director
df['director'] = df['crew'].apply(find_director)
#convert the names into lower case and join string
df['cast'] = df['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
df['director'] = df['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))

df = df.drop(columns=['crew'])
for i in df.index:
    for g in df['cast'][i]:
        if g not in profile:
            profile.append(g)
    if df['director'][i] not in profile:
        profile.append(df['director'][i])
profile_df = pd.DataFrame(profile, columns=["profile"])
profile_df.to_csv('profile.csv', index=False)

In [ ]:
df

In [ ]:
#df['features'] = df['genres'].astype(str) + df['cast'].astype(str) + df['director']

In [6]:
def convert_to_flat(temp):
    flat_list = []
    for i in temp[1]:
        flat_list.append(i)
    for j in temp[2]:
        flat_list.append(j)
    flat_list.append(temp[3])    
    return flat_list

In [7]:
features = []
for index, row in df.iterrows():
    temp = row.to_list()
    flat_list = convert_to_flat(temp)
    features.append(flat_list)
df['features'] = features
df['features'] = df['features'].astype(str)

In [8]:
df

,id,genres,cast,director,features
0,862,"[Animation, Comedy, Family]","[tomhanks, timallen, donrickles]",johnlasseter,"['Animation', 'Comedy', 'Family', 'tomhanks', ..."
1,8844,"[Adventure, Fantasy, Family]","[robinwilliams, jonathanhyde, kirstendunst]",joejohnston,"['Adventure', 'Fantasy', 'Family', 'robinwilli..."
2,15602,"[Romance, Comedy]","[waltermatthau, jacklemmon, ann-margret]",howarddeutch,"['Romance', 'Comedy', 'waltermatthau', 'jackle..."
3,31357,"[Comedy, Drama, Romance]","[whitneyhouston, angelabassett, lorettadevine]",forestwhitaker,"['Comedy', 'Drama', 'Romance', 'whitneyhouston..."
4,11862,[Comedy],"[stevemartin, dianekeaton, martinshort]",charlesshyer,"['Comedy', 'stevemartin', 'dianekeaton', 'mart..."
...,...,...,...,...,...
45533,439050,"[Drama, Family]","[leilahatami, kouroshtahami, elhamkorda]",hamidnematollah,"['Drama', 'Family', 'leilahatami', 'kouroshtah..."
45534,111109,[Drama],"[angelaquino, perrydizon, hazelorencio]",lavdiaz,"['Drama', 'angelaquino', 'perrydizon', 'hazelo..."
45535,67758,"[Action, Drama, Thriller]","[erikaeleniak, adambaldwin, juliedupage]",markl.lester,"['Action', 'Drama', 'Thriller', 'erikaeleniak'..."
45536,227506,[],"[iwanmosschuchin, nathalielissenko, pavelpavlov]",yakovprotazanov,"['iwanmosschuchin', 'nathalielissenko', 'pavel..."


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
tf_idf = TfidfVectorizer()
tf_idf_result = tf_idf.fit_transform(df.features)
sim = cosine_similarity(tf_idf_result)
sim_df = pd.DataFrame(sim)

In [13]:
sim_df

,0,1,2,3,4,5,6,7,8,9,...,45528,45529,45530,45531,45532,45533,45534,45535,45536,45537
0,1.000000,0.040437,0.014214,0.014633,0.015325,0.000000,0.015525,0.039502,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.035121,0.000000,0.000000,0.0,0.0
1,0.040437,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.072366,0.034061,0.036247,...,0.000000,0.0,0.0,0.0,0.000000,0.034213,0.000000,0.000000,0.0,0.0
2,0.014214,0.000000,1.000000,0.038771,0.015100,0.000000,0.041135,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.025023,0.000000,0.000000,0.000000,0.0,0.0
3,0.014633,0.000000,0.038771,1.000000,0.015544,0.010447,0.042346,0.009076,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.035700,0.008070,0.008270,0.007986,0.0,0.0
4,0.015325,0.000000,0.015100,0.015544,1.000000,0.000000,0.016492,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45533,0.035121,0.034213,0.000000,0.008070,0.000000,0.008714,0.000000,0.040993,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.008292,1.000000,0.006899,0.006661,0.0,0.0
45534,0.000000,0.000000,0.000000,0.008270,0.000000,0.008930,0.000000,0.007759,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.008497,0.006899,1.000000,0.006827,0.0,0.0
45535,0.000000,0.000000,0.000000,0.007986,0.000000,0.051770,0.000000,0.027188,0.037683,0.040101,...,0.017216,0.0,0.0,0.0,0.029775,0.006661,0.006827,1.000000,0.0,0.0
45536,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.0,0.0


In [47]:
check_lst = df.id.to_list()
given = []
neighbor_dict = {}

In [48]:
def find_best_k_except_self(l,k,s):
    # find best k in l 
    result = []
    avoid = []
    for _ in range(k):
        min_value = np.NINF
        min_index = -1
        for i in range(len(l)):
            if i not in avoid and l[i] > min_value and i != s:
                min_value = l[i]
                min_index = i
        if min_index != -1:
            result.append(min_index)
            avoid.append(min_value)
    return result

In [49]:
def predictionv2(user_id, movie_id):
    global given
    user_ratings = given[given['userId'] == user_id]
    movie_lst = user_ratings['movieId'].to_list()
    user_avg = user_ratings['rating'].mean()
    movie_index = check_lst.index(movie_id)
    sim = sim_df[movie_index]
    
    neighbors = sim_df.iloc[movie_index].nlargest(6).index.tolist()
    if movie_index in neighbors:
        neighbors.remove(movie_index)
    
    top,bottom = 0,0
    for n in neighbors:
        n_id = check_lst[n]
        if n_id in movie_lst:
            top += sim[n] * (user_ratings[user_ratings['movieId'] == n_id].iloc[0].rating - user_avg)
#         top += sim[n] * user_ratings[user_ratings['movieId'] == n].iloc[0].rating
        bottom += sim[n]
        
    if bottom == 0:
        prediction = user_avg
    
    else:
        prediction = user_avg + top/bottom
    
    return prediction

In [50]:
def get_rmse(test_file,k):
    global given
    testing = pd.read_csv(test_file)
    given = testing.groupby(by="userId").head(k)
    testing = testing[~testing.index.isin(given.index)]
    np_testing = testing.to_numpy()
    ms = 0
    for i in tqdm(range(len(np_testing))):
        userId = int(np_testing[i][0])
        movieId = int(np_testing[i][1])
        rating = np_testing[i][2]
        prediction = predictionv2(userId,movieId)
        ms += (prediction-rating)**2
    return math.sqrt(ms/len(np_testing))

In [51]:
get_rmse("processed/testing-10.csv",10)

1.0146159099259364

In [38]:
get_rmse("processed/testing-20.csv",20)

0.9768510192853541

In [39]:
get_rmse("processed/testing-30.csv",30)

0.9704852985078563

In [40]:
get_rmse("processed/testing-40.csv",40)

0.9629595981039587

In [41]:
get_rmse("processed/testing-50.csv",50)

0.9516342004167937

In [42]:
get_rmse("processed/testing-60.csv",60)

0.9637953632501727

In [43]:
get_rmse("processed/testing-70.csv",70)

0.9727135712212289

In [44]:
get_rmse("processed/testing-80.csv",80)

0.9773877573129336

In [45]:
get_rmse("processed/testing-90.csv",90)

0.9632791552388593

In [46]:
get_rmse("processed/testing-100.csv",100)

0.9795762349583117